### Proyecto 2 - Procesamiento de datos utilizando Apache Hive y Apache Zeppelin

#### Análisis de logs de un web server

![Webserver Logo](https://www.freepngimg.com/thumb/computer/58645-web-http-servers-computer-linux-apache-installation.png) 

Este proyecto demostrará lo fácil que es realizar análisis de logs de servidores web con Apache Hive.

El análisis de logs de un servidor es un caso de uso ideal para Apache Hive.  Es una fuente de datos muy grande y común, y contiene un rico conjunto de información.  Hive le permite almacenar sus logs en archivos en disco de forma económica, a la vez que proporciona una forma rápida y sencilla de realizar análisis de datos sobre ellos. Le mostraremos cómo utilizar Apache Hive en logs de producción basados en texto del mundo real y aprovechar al máximo la potencia de esos datos.  Los datos de logs provienen de muchas fuentes, como servidores web, de archivos y de computación, logs de aplicaciones, contenido generado por el usuario, y pueden utilizarse para monitorizar servidores, mejorar la inteligencia de negocio y de clientes, y mucho más.

#### Paso 1 - Descarga de archivos de datos que vamos a procesar

In [ ]:
# Entrar al directorio
cd /home/dataengineer/files

# Descargamos de la red los archivos que vamos a utilizar
wget https://raw.githubusercontent.com/hivesample/sample/main/Web_log

# Luego copiamos los archivos a HDFS
hdfs dfs -copyFromLocal Web_log /user/dataengineer/filesdata


#### Paso 2 - Iniciar Hadoop, Hive y Zeppelin

In [ ]:
# Buscamos el directorio donde realizamos la instalación de hadoop e ingresamos
cd /home/dataengineer/hadoop/hadoop-3.3.2/

# Iniciamos el sistema de archivos distribuidos
sbin/start-dfs.sh

# Iniciamos yarn
sbin/start-yarn.sh

# Buscamos el directorio donde realizamos la instalacion de hive e ingresamos
cd /home/dataengineer/apachehive/apache-hive-3.1.2-bin/

# Ejecutamos hive
bin/hiveserver2

# Buscamos el directorio donde realizamos la instalacion de zeppelin e ingresamos
cd /home/dataengineer/zeppelin/zeppelin-0.10.1-bin-all/ 

# Ejecutamos zeppelin
bin/zeppelin-daemon.sh start

# Desde el navegador ingresamos a la interfaz gráfica de zeppelin
https://localhost:8080/

#### Paso 3 - Configuración del intérprete Hive en la interfaz gráfica de Apache Zeppelin

In [ ]:
# Esto lo explicamos en el Proyecto 1

#### Paso 4 - Creación de tablas y carga de datos en el notebook Zepellin

In [ ]:
# Creamos la tabla 'apachelog'

%Hive

CREATE TABLE apachelog (
HOST STRING,  
IDENTITY STRING,  
WUSER STRING,  
WTIME STRING,  
request STRING,  
status STRING,  
SIZE STRING) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES 
("input.regex" = "([^ ]*) ([^ ]*) ([^ ]*) (-|\\[[^\\]]*\\]) ([^ \"]*|\"[^\"]*\") (-|[0-9]*) (-|[0-9]*)?")
STORED AS TEXTFILE;

In [ ]:
# Carga de datos

%Hive

LOAD DATA INPATH '/user/dataengineer/filesdata/Web_log' INTO TABLE apachelog;

In [ ]:
# Lanzamos una consulta para verificar que los datos se hayan cargado correctamente

%Hive

SELECT * FROM apachelog;

Si estás familiarizado con los servidores web, reconocerás de que se habla en [Common Log Format](https://www.w3.org/Daemon/User/Config/Logging.html#common-logfile-format). Los campos son:

_remotehost rfc931 authuser [date] "request" status bytes_

| campo         | significado                                                                                                  |
| ------------- | -------------------------------------------------------------------------------------------------------------|
| _remotehost_  | Nombre de host remoto (o número IP si el nombre de host DNS no está disponible).                             |
| _rfc931_      | El logname remoto del usuario. Este campo no nos interesa.                                                   |
| _authuser_    | El username del usuario remoto, autenticado por el servidor HTTP. Este campo no nos interesa.                |
| _[date]_      | Fecha y hora de la petición.                                                                                 |
| _"request"_   | La petición, exactamente como vino del navegador o del cliente.                                              |
| _status_      | El código de estado HTTP que el servidor envió de vuelta al cliente.                                         |
| _bytes_       | El número de bytes (`Content-Length`) transferidos al cliente.                                               |

A continuación, tenemos que dividirlo en columnas individuales. Usaremos la función especial incorporada [regexp\_extract()]
para realizar el análisis. Esta función compara una columna con una expresión regular con uno o más [grupos de captura](https://www.tutorialspoint.com/scala/scala_regular_expressions.htm) y le permite extraer uno de los grupos coincidentes. Utilizaremos una expresión regular para cada campo que deseemos extraer.

Sitio web donde existe material para comenzar a estudiar las Expresiones regulares (https://www.tutorialspoint.com/scala/scala_regular_expressions.htm). 

#### Paso 5 - Análisis de datos

##### Estadísticas sobre el tamaño de los contenidos

Calculemos algunas estadísticas sobre el tamaño de los contenidos devueltos por el servidor web. En concreto, nos gustaría saber cuál es el tamaño medio, mínimo y máximo de los contenidos.

In [ ]:
%Hive

select 
min(size) AS Minimum,
max(size) AS Maximum,
avg(size) AS Average
from apachelog;

##### HTTP Status Analisis

A continuación, vamos a ver los valores de status que aparecen en el log. Queremos saber qué valores de status aparecen en los datos y cuántas veces.

In [ ]:
%Hive

select 
status,
count(status) as countbystatus
from apachelog group by status order by status asc;

##### Hosts Frecuentes

Veamos los hosts que han accedido al servidor con frecuencia. Al igual que con el análisis del código de respuesta 

A continuación, filtramos el resultado en función del recuento de accesos de cada host.  A continuación, seleccionamos la columna 'host' y mostramos algunos elementos del resultado.

In [ ]:
%Hive

select 
host, 
count(host)  as CountofHost 
from apachelog group by Host order by CountofHost desc;  

##### Top Paths

Para el ejemplo final, encontraremos las top paths (URIs) en el log.

In [ ]:
%Hive

select 
regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1) as path, 
count(regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1)) as Countofpath 
from apachelog group by regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1)  order by Countofpath desc;  

##### Las diez principales error paths

¿Cuáles son los diez paths principales que no tienen el código de retorno 200?

In [ ]:
%Hive

select 
regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1) as path, 
count(regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1)) as Countofpath, 
status
from apachelog 
where status != 200
group by regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1),status
order by Countofpath desc limit 10;  

##### Número de hosts únicos

¿Cuántos hosts únicos hay en todo el log?

In [ ]:
%Hive

select
distinct(Host)  as Host
from apachelog; 

In [ ]:
%Hive

select 
count(distinct(Host)) as CountofUniqueHost
from apachelog; 

##### Número de hosts únicos diarios

Como ejercicio avanzado, vamos a determinar el número de hosts únicos en todo el log día a día. Este cálculo nos dará el número de hosts únicos diarios. 

**Descripción de cada variable

| column | explanation          |
| ------ | -------------------- |
| `host` | the host name        |
| `day`  | the day of the month |


| column  | explanation                                        |
| ------- | -------------------------------------------------- |
| `day`   | the day of the month                               |
| `count` | the number of unique requesting hosts for that day |

##### Número de requests diarias por host único

A continuación, vamos a determinar el número de peticiones por día.  Queremos una lista por día del mes en aumento y el promedio asociado de peticiones por host para ese día.

In [ ]:
%Hive


select
host,
count(request)  as NumberofRequest,
split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0] as datelimited
from 
apachelog group by split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0], Host order by split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0] asc

##### Exploración de los códigos de estado 404

Vamos a profundizar y explorar los registros de estado de error 404. Todos hemos visto esas páginas web "404 Not Found". Los errores 404 se devuelven cuando el servidor no puede encontrar el recurso (página u objeto) que el navegador o el cliente solicitó.

##### Contando Códigos de Respuesta 404

In [ ]:
%Hive 

select
status,
count(status) as countofstatus
from apachelog where status = 404 group by status;

##### Listado de las veinte principales paths con código de respuesta 404

Imprime una lista de las veinte paths que generan más errores 404.

*Recuerde, las paths principales deben estar ordenadas*.

In [ ]:
%Hive

select
regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1) as path, 
status,
count(status)
from apachelog where status = 404 group by status, regexp_extract(request, '^.*"\\w+\\s+([^\\s]+)\\s+HTTP.*"', 1) order by count(status) desc limit 20;


##### Visualización de los errores 404 por día

Imprimir gráfico de los códigos de respuesta 404 por día.

In [ ]:
%Hive

select
status,
count(status) as countofstatus,
split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0] as datelimited
from 
apachelog where status = 404
group by split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0], status order by split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0];


##### Los cinco días con más errores 404

¿cuáles son los cinco días con más errores 404 y el número correspondiente de errores 404?

In [ ]:
%Hive

select
status,
count(status) as countofstatus,
split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0] as datelimited
from 
apachelog where status = 404
group by split(regexp_extract(WTIME, '^.*\\[(\\d\\d/\\w{3}/\\d{4}:\\d{2}:\\d{2}:\\d{2} -\\d{4})]', 1),':')[0], status order by count(status) desc limit 5;